# 3DES / Triple DES (TDEA)

3DES (Triple DES), also known as TDEA, is a legacy symmetric block cipher derived from DES. It encrypts 64-bit blocks by applying the DES cipher three times using an Encrypt–Decrypt–Encrypt (E–D–E) sequence. Although it was originally designed to extend the life of DES, 3DES is now largely replaced by AES in modern systems.

---

## How It Works

- **3-Key Variant:**  
  Uses three independent DES keys (K1, K2, K3).  
  **Encryption:**  
  ```text
  C = E_{K3}( D_{K2}( E_{K1}( P ) ) )
  ```
  
- **2-Key Variant:**  
  Uses two DES keys (K1, K2), where K1 is reused.  
  **Encryption:**  
  ```text
  C = E_{K1}( D_{K2}( E_{K1}( P ) ) )
  ```

_Decryption simply reverses these steps._

---

## Parameters

- **Block Size:** 64 bits (8 bytes)
- **Key Options:**
  - **3-Key Variant:**  
    - Three 56‑bit DES keys (stored in 64-bit values with parity)  
    - Provides roughly 112 bits of security (subject to meet-in-the-middle attacks).
  - **2-Key Variant:**  
    - Two 56‑bit DES keys reused as K1/K3  
    - Provides about 80 bits of security.
- **Modes of Operation:**  
  Common modes include CBC, CTR, CFB, and OFB.  
  Do not use ECB mode—prefer AEAD modes when possible.

---

## Security Considerations

- **Deprecated Standard:**  
  3DES is considered legacy per NIST guidelines—AES is the modern standard.
- **Block Size Vulnerabilities:**  
  A 64-bit block size can lead to birthday attacks when encrypting large volumes of data under one key/IV.
- **Keying Option Choice:**  
  If 3DES must be used, the 3-key variant is preferred over the 2-key variant for improved security.

---

## Usage History

- **Legacy Financial Systems:**  
  Previously deployed in older payment processing and financial systems.
- **Hardware Security Modules (HSMs):**  
  Often found in systems requiring backward compatibility.
- **Compatibility in Protocols:**  
  Still used for historical and transitional support in

In [3]:
from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

def strong_des3_key(num_bytes: int) -> bytes:
    """
    Generate a DES3 key of length num_bytes (16 for 2-key, 24 for 3-key),
    adjust parity, and ensure it's not a weak or invalid key.
    """
    assert num_bytes in (16, 24)
    while True:
        k = get_random_bytes(num_bytes)
        k = DES3.adjust_key_parity(k)
        try:
            DES3.new(k, DES3.MODE_ECB)  # validate key
            return k
        except ValueError:
            continue

# --- Choose ONE of the two key options ---

# (A) 3-key 3DES (recommended if you must use 3DES)
key = strong_des3_key(24)   # 24 bytes → K1||K2||K3

# (B) 2-key 3DES (uncomment to use)
# key = strong_des3_key(16) # 16 bytes → K1||K2 (EDE with K1,K2,K1)

# --- CBC needs an 8-byte IV ---
iv = get_random_bytes(8)

# --- Encrypt ---
cipher_enc = DES3.new(key, DES3.MODE_CBC, iv=iv)
plaintext  = b"3DES: CBC with padding"
ciphertext = cipher_enc.encrypt(pad(plaintext, block_size=8))

print("Key length:", len(key), "bytes")
print("IV (hex):", iv.hex())
print("Ciphertext (hex):", ciphertext.hex())

# --- Decrypt ---
cipher_dec = DES3.new(key, DES3.MODE_CBC, iv=iv)
recovered  = unpad(cipher_dec.decrypt(ciphertext), block_size=8)

print("Recovered:", recovered)
assert recovered == plaintext


Key length: 24 bytes
IV (hex): 5931f76766271547
Ciphertext (hex): f802ed6843373e8626eeb6481367a8d5a73c40c409e1a89b
Recovered: b'3DES: CBC with padding'
